In [270]:
import numpy as np
import pandas as pd
import random
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from tqdm import tqdm
import time
import pickle
from functools import reduce
from deap import creator, base, tools, algorithms, benchmarks, cma
import hashlib
from itertools import chain

%matplotlib inline
import matplotlib.pyplot as plt

In [271]:
INPUT_PATH = '../../2/limontov/input/'
np.random.seed(6)
CHECKPOINT_FREQ = 10
storage = {}

In [272]:
def read_base(N, map_f=None):
    global storage
    dataset = storage.get(N)
    c_type = 'mapped' if map_f else 'raw'
    if dataset:
        if dataset.get(c_type):
            print('found cached data...')
            return dataset[c_type]
    else:
        storage[N] = dict()
    print('reading data from source...')
    LINES = 10**6
    path = '{}Base{}.txt'.format(INPUT_PATH, N)
    X, y = [], []
    with open(path, 'r') as fin:
        for l in tqdm(fin.readlines()):
            x = l.split()[0]
            if map_f:
                X.append(map_f(x))
            else:
                X.append(list(x))
            y.append(l.split()[1])
    print('converting to dataframe...')
    storage[N][c_type] = (pd.DataFrame(X, dtype=int), pd.Series(y, dtype=int))
    return storage[N][c_type]

In [273]:
# X, y = read_base(16)
# print(X[:5])
# print(y[:5])

In [274]:
def do_xor(x):
    cur_xor = 0
    res = []
    for i in x:
        cur_xor ^= int(i)
        res.append(cur_xor)
    return res

def do_hash(ch):
    return list(''.join(np.array([format(x, 'b').rjust(8, '0').split() for x in bytearray(hashlib.sha256(ch.encode()).digest())]).flatten()))

In [275]:
def learn(X, y, model=None, N=None):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    if model is None:
        filename = '../../2/limontov/output/pickled_{}.pkl'.format(N)
        with open(filename, 'rb') as fin:
            model = pickle.load(fin)
    else:
        model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    score = model.score(X_test, y_test)
    print(score)
    return model, score

In [306]:
X, y = read_base(48, do_hash)
N = 48

found cached data...


In [203]:
# do_hash('01110101')

In [204]:
X_try, y_try = read_base(8, do_hash)
X_try.describe()
X_try[:5]




  0%|          | 0/1000000 [00:00<?, ?it/s]

reading data from source...





  0%|          | 838/1000000 [00:00<01:59, 8378.33it/s]


  0%|          | 1915/1000000 [00:00<01:51, 8975.86it/s]


  0%|          | 3268/1000000 [00:00<01:39, 9983.53it/s]


  0%|          | 4365/1000000 [00:00<01:37, 10258.05it/s]


  1%|          | 5503/1000000 [00:00<01:38, 10142.78it/s]


  1%|          | 6943/1000000 [00:00<01:29, 11128.63it/s]


  1%|          | 8291/1000000 [00:00<01:24, 11742.52it/s]


  1%|          | 9877/1000000 [00:00<01:17, 12733.56it/s]


  1%|          | 11278/1000000 [00:00<01:15, 13090.43it/s]


  1%|▏         | 12811/1000000 [00:01<01:12, 13690.15it/s]


  1%|▏         | 14192/1000000 [00:01<01:14, 13181.11it/s]


  2%|▏         | 15550/1000000 [00:01<01:14, 13297.90it/s]


  2%|▏         | 16959/1000000 [00:01<01:12, 13524.80it/s]


  2%|▏         | 18319/1000000 [00:01<01:13, 13345.24it/s]


  2%|▏         | 19659/1000000 [00:01<01:16, 12885.36it/s]


  2%|▏         | 21002/1000000 [00:01<01:15, 13043.71it/s]


  2%|▏         | 22312/1000000 [0

 16%|█▌        | 155708/1000000 [00:15<01:07, 12493.86it/s]


 16%|█▌        | 156963/1000000 [00:15<01:11, 11720.79it/s]


 16%|█▌        | 158178/1000000 [00:15<01:11, 11846.24it/s]


 16%|█▌        | 159502/1000000 [00:15<01:08, 12231.73it/s]


 16%|█▌        | 160860/1000000 [00:15<01:06, 12602.60it/s]


 16%|█▌        | 162247/1000000 [00:15<01:04, 12956.65it/s]


 16%|█▋        | 163720/1000000 [00:15<01:02, 13441.27it/s]


 17%|█▋        | 165115/1000000 [00:15<01:04, 12851.38it/s]


 17%|█▋        | 166516/1000000 [00:15<01:03, 13178.00it/s]


 17%|█▋        | 167854/1000000 [00:15<01:02, 13236.27it/s]


 17%|█▋        | 169194/1000000 [00:16<01:02, 13283.35it/s]


 17%|█▋        | 170547/1000000 [00:16<01:02, 13355.52it/s]


 17%|█▋        | 171950/1000000 [00:16<01:01, 13549.55it/s]


 17%|█▋        | 173329/1000000 [00:16<01:00, 13619.87it/s]


 17%|█▋        | 174694/1000000 [00:16<01:03, 13016.42it/s]


 18%|█▊        | 176033/1000000 [00:16<01:02, 13123.78it/s]


 18%|█▊ 

 31%|███       | 306036/1000000 [00:27<00:58, 11938.58it/s]


 31%|███       | 307295/1000000 [00:28<00:57, 12125.20it/s]


 31%|███       | 308515/1000000 [00:28<00:58, 11872.21it/s]


 31%|███       | 309708/1000000 [00:29<03:42, 3105.46it/s] 


 31%|███       | 310840/1000000 [00:29<02:53, 3968.04it/s]


 31%|███       | 311904/1000000 [00:29<02:20, 4887.20it/s]


 31%|███▏      | 312929/1000000 [00:29<01:58, 5796.94it/s]


 31%|███▏      | 314373/1000000 [00:29<01:37, 7065.34it/s]


 32%|███▏      | 315672/1000000 [00:29<01:23, 8184.81it/s]


 32%|███▏      | 316993/1000000 [00:29<01:13, 9238.59it/s]


 32%|███▏      | 318265/1000000 [00:29<01:07, 10064.88it/s]


 32%|███▏      | 319711/1000000 [00:29<01:01, 11073.97it/s]


 32%|███▏      | 321057/1000000 [00:30<00:58, 11694.96it/s]


 32%|███▏      | 322456/1000000 [00:30<00:55, 12299.77it/s]


 32%|███▏      | 323853/1000000 [00:30<00:53, 12755.84it/s]


 33%|███▎      | 325210/1000000 [00:30<00:52, 12960.16it/s]


 33%|███▎     

...

 60%|██████    | 602309/1000000 [00:57<01:05, 6065.07it/s]


 60%|██████    | 603412/1000000 [00:58<00:58, 6771.58it/s]


 60%|██████    | 604693/1000000 [00:58<00:50, 7886.76it/s]


 61%|██████    | 605903/1000000 [00:58<00:44, 8806.24it/s]


 61%|██████    | 607159/1000000 [00:58<00:40, 9671.58it/s]


 61%|██████    | 608363/1000000 [00:58<00:38, 10277.08it/s]


 61%|██████    | 609550/1000000 [00:58<00:36, 10707.78it/s]


 61%|██████    | 610749/1000000 [00:58<00:35, 11053.12it/s]


 61%|██████    | 611939/1000000 [00:58<00:35, 11017.87it/s]


 61%|██████▏   | 613126/1000000 [00:58<00:34, 11260.25it/s]


 61%|██████▏   | 614303/1000000 [00:58<00:33, 11407.48it/s]


 62%|██████▏   | 615534/1000000 [00:59<00:32, 11663.63it/s]


 62%|██████▏   | 616723/1000000 [00:59<00:33, 11508.27it/s]


 62%|██████▏   | 617971/1000000 [00:59<00:32, 11782.16it/s]


 62%|██████▏   | 619342/1000000 [00:59<00:30, 12300.75it/s]


 62%|██████▏   | 620588/1000000 [00:59<00:32, 11634.99it/s]


 62%|██████▏ 

 75%|███████▌  | 750872/1000000 [01:10<00:19, 12648.34it/s]


 75%|███████▌  | 752155/1000000 [01:10<00:19, 12517.56it/s]


 75%|███████▌  | 753518/1000000 [01:10<00:19, 12831.17it/s]


 75%|███████▌  | 754812/1000000 [01:10<00:19, 12503.89it/s]


 76%|███████▌  | 756330/1000000 [01:10<00:18, 13201.47it/s]


 76%|███████▌  | 757784/1000000 [01:10<00:17, 13574.54it/s]


 76%|███████▌  | 759157/1000000 [01:10<00:17, 13508.52it/s]


 76%|███████▌  | 760655/1000000 [01:10<00:17, 13917.95it/s]


 76%|███████▌  | 762058/1000000 [01:10<00:17, 13813.70it/s]


 76%|███████▋  | 763447/1000000 [01:10<00:17, 13339.43it/s]


 76%|███████▋  | 764790/1000000 [01:11<00:17, 13366.20it/s]


 77%|███████▋  | 766133/1000000 [01:11<00:17, 13250.97it/s]


 77%|███████▋  | 767470/1000000 [01:11<00:17, 13285.43it/s]


 77%|███████▋  | 768873/1000000 [01:11<00:17, 13499.15it/s]


 77%|███████▋  | 770266/1000000 [01:11<00:16, 13622.18it/s]


 77%|███████▋  | 771631/1000000 [01:11<00:17, 12940.96it/s]


 77%|███

 91%|█████████ | 908338/1000000 [01:24<00:07, 12441.87it/s]


 91%|█████████ | 909771/1000000 [01:24<00:06, 12952.12it/s]


 91%|█████████ | 911141/1000000 [01:24<00:06, 13165.84it/s]


 91%|█████████▏| 912589/1000000 [01:24<00:06, 13533.50it/s]


 91%|█████████▏| 913961/1000000 [01:24<00:06, 12657.82it/s]


 92%|█████████▏| 915312/1000000 [01:24<00:06, 12900.31it/s]


 92%|█████████▏| 916901/1000000 [01:25<00:06, 13671.55it/s]


 92%|█████████▏| 918294/1000000 [01:25<00:06, 13505.51it/s]


 92%|█████████▏| 919776/1000000 [01:25<00:05, 13872.29it/s]


 92%|█████████▏| 921179/1000000 [01:25<00:05, 13539.93it/s]


 92%|█████████▏| 922546/1000000 [01:25<00:06, 12811.55it/s]


 92%|█████████▏| 923856/1000000 [01:25<00:05, 12894.35it/s]


 93%|█████████▎| 925394/1000000 [01:25<00:05, 13550.42it/s]


 93%|█████████▎| 926880/1000000 [01:25<00:05, 13917.66it/s]


 93%|█████████▎| 928287/1000000 [01:25<00:06, 10516.76it/s]


 93%|█████████▎| 929473/1000000 [01:26<00:09, 7557.88it/s] 


 93%|███

converting to dataframe...


,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
0,0,0,1,0,0,0,1,1,1,0,...,1,1,0,1,0,1,1,1,1,0
1,0,0,1,0,1,1,1,1,1,1,...,1,1,1,0,1,0,1,1,1,0
2,0,0,1,1,1,0,1,1,0,0,...,0,0,1,0,0,1,0,1,0,0
3,1,0,1,1,1,0,0,1,0,1,...,0,0,0,0,1,0,1,0,1,1
4,0,1,1,0,1,1,0,0,0,1,...,0,1,0,0,1,1,1,1,1,0


In [ ]:
# g = list(map(bool, [np.random.randint(0, 2) for i in range(256)]))
# X_try.values[:, g]
# print(X_try.values[:, g].shape)
# X_try[[0, 2, 3, 4, 6]]

In [ ]:
# X, y = read_base(48, do_hash)
# model = LogisticRegression(n_jobs=-1, random_state=6)

In [ ]:
# pd.concat([pd.DataFrame(np.ones((3, 5)), dtype=int), pd.DataFrame(np.ones((3, 2)), dtype=int)], axis=1)

In [317]:
def get_fitness(x):
    model = LogisticRegression(n_jobs=-1, random_state=7)
    x = np.array(x)
    mask = np.sign(x)
    m = []
    fl = False
    print('Getting first {} valuable bits'.format(N))
    for idx, i in enumerate(mask):
        if i == 1:
            m.append(idx)
        if len(m) == N:
            break
    else:
        fl = True
    
    print('Generating final X')
    print('m', m)
    if fl:
        print('Bad concat')
        X_masked = pd.concat([X[m], pd.DataFrame(np.ones((X.shape[0], N - len(m))))], axis=1)
    else:
        print('Could find needed bits')
        X_masked = X[m]
    print('Applying XOR')
    X_masked = X_masked.apply(do_xor, axis=0)
    print('Learning the model')
    m, score = learn(X_masked, y, model)
    print('Model learned')
    return (score, )

In [313]:
y1 = np.array([1, 0, 1, 0])
y2 = np.array([1, 1, 0, 0])
np.sum(1 - np.abs(y1 - y2))

2

In [314]:
# ?cma.Strategy

In [ ]:
def learn_gen():
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)

    toolbox = base.Toolbox()
    toolbox.register("evaluate", benchmarks.rastrigin)

    gen_size = 256
    strategy = cma.Strategy(centroid=[0.0] * gen_size, sigma=1.0, lambda_=5)
    toolbox.register("generate", strategy.generate, creator.Individual)
    toolbox.register("evaluate", get_fitness)
    toolbox.register("update", strategy.update)

    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    stats.register("max", np.max)

    # algorithms.eaGenerateUpdate(toolbox, ngen=gen_size, stats=stats, halloffame=hof)

    ngen = 200
    start_gen = 0
        
    for gen in range(start_gen, ngen):
        print('Generating a new population')
        population = toolbox.generate()
        print('Generated population of {} species'.format(len(population)))
        print('Clipping')
        for p in population:
            for i in range(len(p)):
                p[i] = np.clip(p[i], -2, 2)
        print('Evaluating the individuals')
        fitnesses = toolbox.map(toolbox.evaluate, population)
        for ind, fit in zip(population, fitnesses):
            ind.fitness.values = fit
        print('Updating HOF population')
        hof.update(population)

        print('Updating population')
        toolbox.update(population)
        print("Gettings stats")
        print(stats.compile(population))
    
    population = toolbox.generate()[0]
    print(population)
    return population, hof

In [320]:
def learn_gen():
    
    def import_checkpoint(gen_num):
        with open('checkpoints/cp_{}.pkl'.format(gen_num), "rb") as cp_file:
            cp = pickle.load(cp_file)
            return cp
    
    def export_checkpoint(gen_num, **kwargs):
        cp = dict(population=population, generation=gen, halloffame=hof,
                  logbook=logbook, rndstate=random.getstate())
        with open("checkpoints/cp_{}.pkl".format(gen_num), "wb") as cp_file:
            pickle.dump(kwargs, cp_file)
        
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)

    toolbox = base.Toolbox()
    toolbox.register("evaluate", benchmarks.rastrigin)

    gen_size = 256
    strategy = cma.Strategy(centroid=[0.0] * gen_size, sigma=1.0, lambda_=5)
    toolbox.register("generate", strategy.generate, creator.Individual)
    toolbox.register("evaluate", get_fitness)
    toolbox.register("update", strategy.update)

    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)

    # algorithms.eaGenerateUpdate(toolbox, ngen=gen_size, stats=stats, halloffame=hof)

    logbook = tools.Logbook()
    logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])
    ngen = 200
    start_gen = 0
    if start_gen:
        cp = import_checkpoint(start_gen)
        population = cp['population']
        toolbox.update(population)
        start_gen = cp["generation"]
        hof = cp["halloffame"]
        logbook = cp["logbook"]
        random.setstate(cp["rndstate"])
        
    for gen in range(start_gen, ngen):
        print('Generating a new population')
        population = toolbox.generate()
        print('Generated population of {} species'.format(len(population)))
        print('Clipping')
        for p in population:
            for i in range(len(p)):
                p[i] = max(p[i], -10)
                p[i] = min(p[i], 10)
        print('Evaluating the individuals')
        fitnesses = toolbox.map(toolbox.evaluate, population)
        for ind, fit in zip(population, fitnesses):
            ind.fitness.values = fit
        print('Updating HOF population')
        hof.update(population)

        print('Updating population')
        toolbox.update(population)
        print("Gettings stats")
        record = stats.compile(population) if stats is not None else {}
        logbook.record(gen=gen, nevals=len(population), **record)
        print(logbook.stream)
        if gen % CHECKPOINT_FREQ == 0:
            export_checkpoint(gen, population=population, generation=gen, halloffame=hof, logbook=logbook, rndstate=random.getstate())
    population = toolbox.generate()[0]
    print(population)

In [321]:
learn_gen()

/Users/chiselko6/anaconda/envs/iai/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/Users/chiselko6/anaconda/envs/iai/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


Generating a new population
Generated population of 5 species
Clipping
Evaluating the individuals
Getting first 48 valuable bits
Generating final X
m [0, 2, 5, 6, 7, 8, 9, 10, 11, 12, 14, 18, 19, 20, 21, 22, 25, 26, 34, 35, 37, 38, 40, 41, 46, 48, 52, 55, 56, 61, 62, 64, 66, 67, 68, 73, 75, 77, 81, 85, 86, 88, 90, 92, 93, 96, 102, 103]
Could find needed bits
Applying XOR
Learning the model


/Users/chiselko6/anaconda/envs/iai/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


0.518065
Model learned
Getting first 48 valuable bits
Generating final X
m [0, 1, 2, 3, 4, 6, 7, 20, 25, 26, 27, 29, 34, 35, 36, 42, 43, 46, 47, 48, 50, 51, 52, 54, 56, 60, 63, 64, 65, 68, 69, 71, 72, 76, 79, 82, 88, 90, 93, 95, 96, 97, 98, 100, 102, 104, 105, 109]
Could find needed bits
Applying XOR
Learning the model
0.518065
Model learned
Getting first 48 valuable bits
Generating final X
m [0, 1, 4, 5, 6, 7, 9, 10, 11, 12, 13, 16, 17, 18, 20, 22, 23, 24, 26, 27, 28, 30, 31, 32, 33, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 51, 53, 58, 60, 63, 64, 70, 71, 76, 77]
Could find needed bits
Applying XOR
Learning the model
0.51807
Model learned
Getting first 48 valuable bits
Generating final X
m [1, 6, 10, 11, 15, 16, 18, 19, 21, 23, 26, 31, 32, 33, 34, 37, 39, 40, 41, 42, 44, 45, 46, 47, 48, 51, 53, 55, 60, 61, 63, 64, 65, 66, 67, 68, 73, 75, 76, 78, 79, 80, 82, 83, 84, 87, 88, 91]
Could find needed bits
Applying XOR
Learning the model
0.518065
Model learned
Getting first 48 val

Applying XOR
Learning the model
0.518065
Model learned
Updating HOF population
Updating population
Gettings stats
4  	5     	0.518065	0    	0.518065	0.518065
Generating a new population
Generated population of 5 species
Clipping
Evaluating the individuals
Getting first 48 valuable bits
Generating final X
m [0, 1, 4, 5, 6, 10, 12, 13, 16, 17, 23, 25, 27, 30, 34, 36, 37, 38, 39, 40, 45, 46, 48, 53, 55, 58, 59, 60, 61, 62, 63, 64, 67, 77, 79, 81, 82, 84, 85, 89, 90, 91, 92, 93, 94, 95, 97, 98]
Could find needed bits
Applying XOR
Learning the model
0.518065
Model learned
Getting first 48 valuable bits
Generating final X
m [0, 1, 4, 5, 6, 7, 10, 11, 12, 13, 14, 16, 17, 20, 23, 25, 26, 27, 36, 37, 38, 39, 40, 41, 42, 44, 45, 48, 53, 58, 59, 60, 61, 63, 64, 67, 71, 72, 75, 77, 79, 80, 82, 84, 85, 86, 90, 91]
Could find needed bits
Applying XOR
Learning the model
0.518065
Model learned
Getting first 48 valuable bits
Generating final X
m [0, 2, 3, 4, 5, 6, 7, 10, 11, 13, 14, 16, 17, 23, 25, 27,

Applying XOR
Learning the model
0.518065
Model learned
Getting first 48 valuable bits
Generating final X
m [1, 4, 8, 9, 11, 12, 13, 14, 16, 17, 20, 23, 24, 25, 34, 36, 37, 39, 40, 45, 48, 50, 51, 53, 55, 56, 58, 60, 61, 62, 63, 64, 72, 74, 77, 79, 82, 84, 85, 86, 89, 90, 91, 94, 95, 97, 98, 99]
Could find needed bits
Applying XOR
Learning the model
0.518065
Model learned
Getting first 48 valuable bits
Generating final X
m [1, 4, 5, 11, 12, 13, 16, 17, 23, 24, 25, 26, 30, 32, 33, 34, 36, 37, 38, 39, 40, 44, 45, 47, 48, 50, 51, 53, 55, 56, 57, 58, 60, 61, 62, 63, 64, 69, 75, 77, 79, 82, 84, 85, 86, 89, 90, 94]
Could find needed bits
Applying XOR
Learning the model
0.518065
Model learned
Updating HOF population
Updating population
Gettings stats
9  	5     	0.518065	0    	0.518065	0.518065
Generating a new population
Generated population of 5 species
Clipping
Evaluating the individuals
Getting first 48 valuable bits
Generating final X
m [0, 1, 4, 8, 13, 16, 17, 20, 21, 23, 24, 25, 32, 34, 

Learning the model
0.518065
Model learned
Getting first 48 valuable bits
Generating final X
m [0, 1, 4, 5, 8, 9, 13, 16, 17, 19, 20, 23, 24, 32, 34, 35, 36, 37, 38, 39, 40, 42, 45, 47, 48, 50, 51, 53, 55, 56, 58, 60, 61, 62, 64, 69, 71, 73, 74, 75, 77, 79, 82, 83, 84, 85, 86, 87]
Could find needed bits
Applying XOR
Learning the model
0.518065
Model learned
Getting first 48 valuable bits
Generating final X
m [0, 1, 4, 5, 8, 9, 10, 13, 14, 16, 17, 18, 20, 23, 24, 27, 33, 34, 36, 37, 38, 39, 40, 45, 48, 50, 51, 53, 54, 55, 56, 58, 60, 61, 62, 71, 73, 75, 77, 79, 81, 82, 84, 85, 86, 87, 88, 89]
Could find needed bits
Applying XOR
Learning the model
0.518065
Model learned
Getting first 48 valuable bits
Generating final X
m [0, 1, 4, 5, 8, 9, 14, 16, 17, 20, 24, 32, 34, 36, 37, 38, 39, 40, 42, 45, 46, 48, 50, 51, 53, 54, 56, 58, 60, 61, 62, 63, 64, 70, 71, 73, 75, 77, 79, 82, 83, 84, 85, 86, 87, 88, 89, 90]
Could find needed bits
Applying XOR
Learning the model
0.518065
Model learned
Getting

Applying XOR
Learning the model
0.518065
Model learned
Updating HOF population
Updating population
Gettings stats
18 	5     	0.518065	0    	0.518065	0.518065
Generating a new population
Generated population of 5 species
Clipping
Evaluating the individuals
Getting first 48 valuable bits
Generating final X
m [0, 1, 4, 5, 10, 12, 13, 14, 16, 17, 24, 25, 30, 33, 34, 35, 36, 37, 39, 40, 42, 45, 47, 48, 50, 51, 53, 54, 56, 58, 60, 61, 62, 64, 65, 66, 69, 71, 73, 75, 77, 84, 85, 86, 87, 88, 89, 90]
Could find needed bits
Applying XOR
Learning the model
0.518065
Model learned
Getting first 48 valuable bits
Generating final X
m [1, 4, 5, 9, 10, 12, 13, 14, 16, 17, 22, 24, 26, 27, 30, 33, 34, 35, 36, 37, 39, 40, 42, 44, 45, 47, 48, 50, 51, 53, 54, 55, 56, 58, 60, 61, 62, 63, 71, 73, 74, 75, 77, 78, 80, 84, 85, 86]
Could find needed bits
Applying XOR
Learning the model
0.518065
Model learned
Getting first 48 valuable bits
Generating final X
m [0, 1, 4, 5, 10, 12, 13, 14, 15, 16, 17, 20, 24, 26, 2

KeyboardInterrupt: 

In [311]:
?np.sign